# Introduction

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Woodstock_poster.jpg/800px-Woodstock_poster.jpg)

# Setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/Woodstock")

# Creating the dataframe 

In [ ]:
fridayToSaturday = dfs[1]

fridayToSaturday["Day"] = "Friday, August 15 – Saturday, August 16"

fridayToSaturday

In [ ]:
saturdayToSunday = dfs[2]

saturdayToSunday["Day"] = "Saturday, August 16 – Sunday, August 17"

saturdayToSunday

In [ ]:
sundayToMonday = dfs[3]

sundayToMonday["Day"] = "Sunday, August 17 – Monday, August 18"

sundayToMonday

In [ ]:
df = pd.concat([fridayToSaturday, saturdayToSunday, sundayToMonday], ignore_index=True)

df

# Data cleaning and transformation

In [ ]:
df['Time'] = df['Time'].str.replace('Midnight', '12:00 AM')

In [ ]:
# Splitting the time column
df[['Start Time', 'End Time']] = df['Time'].str.split(' – ', expand=True)

# Convert starting and ending time columns to datetime objects
df['Start Time'] = pd.to_datetime(df['Start Time'], format='%I:%M %p')
df['End Time'] = pd.to_datetime(df['End Time'], format='%I:%M %p')

# Calculate concert duration
df['Concert Duration'] = df['End Time'] - df['Start Time']

In [ ]:
df.head()